In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
from tcvae import predict, dataset, localconfig


In [ ]:
conf = localconfig.LocalConfig()
conf.load_config_from_file(
    "checkpoints/cnn_ae5_2d/Default_cnn_ae5_2d.json"
)
conf.dataset_dir = "complete_dataset"
conf.batch_size = 1

_, _, test = dataset.get_dataset(conf)
test_iter = iter(test)

In [ ]:
_model = predict.load_model(
    conf,
    "checkpoints/cnn_ae5_2d/79_cnn_ae5_2d_0.01495.h5"
)

## Export Some Examples

In [ ]:
import soundfile as sf
import tsms


def write_audio(h_freq, h_mag, h_phase, conf, audio_path):
    audio = tsms.core.harmonic_synthesis(
        h_freq, h_mag, h_phase, conf.sample_rate, conf.frame_size)
    audio = np.squeeze(audio.numpy())
    audio = audio / np.max(np.abs(audio))
    sf.write(audio_path, audio, samplerate=conf.sample_rate)


for i in range(0, 20):
    batch = next(test_iter)
    (h_freq_true, h_mag_true, h_phase_true,
     h_freq_pred, h_mag_pred, h_phase_pred) = predict.get_prediction(_model, batch, conf)

    true_path = os.path.join(os.getcwd(), "predictions", f"{i}_true.wav")
    pred_path = os.path.join(os.getcwd(), "predictions", f"{i}_pred.wav")

    write_audio(h_freq_true, h_mag_true, h_phase_true, conf, true_path)
    write_audio(h_freq_pred, h_mag_pred, h_phase_pred, conf, pred_path)

    print(i)